In [1]:
%matplotlib inline
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

import os
import cv2
import numpy as np
import torch
from torch.autograd import Variable
import quat_math
import pickle
import glob

from PIL import Image
import scipy.io as scio
from functools import partial
from object_pose_utils.utils import to_np, to_var
from object_pose_utils.utils.display import *

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 20, 12
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import warnings
warnings.filterwarnings('ignore')

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


## Dataset Information

In [2]:
dataset_root = '/ssd0/datasets/ycb/YCB_Video_Dataset'

non_sym_objs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 17, 18,]
sym_objs = [13, 16, 19, 20, 21]

with open('{0}/image_sets/classes.txt'.format(dataset_root)) as f:                                    
    classes = f.read().split()
classes.insert(0, '__background__')

In [3]:
def recursiveUpdate(dict_base, dict_new):
    for k,v in dict_new.items():
        if(k in dict_base):
            dict_base[k].update(v)
        else:
            dict_base[k] = v

def updateResults(results_dict, new_results_filename):
    new_data = np.load(new_results_filename, allow_pickle=True)
    new_results = {k:v.item() for k,v in new_data.items()}
    recursiveUpdate(results_dict, new_results)

In [23]:
def non_func(x):
    return x

def mean_abs(x):
    return np.mean(np.abs(x))

def log_clean(x):
    return np.log(np.maximum(1e-6, x))

def mean_log_clean(x):
    return np.mean(log_clean(x))

def mean_log(x):
    return np.mean(np.log(x))   

def makeTableEntries(data_dict, keys = None, objs = list(range(1,22)), 
              val_func = non_func, bold_func = max,
              individual = False,
              agg_title = 'All'):
    if(keys is None):
        keys = list(data_dict.keys())
    
    data_str = ''
    if(individual):
        for obj in objs:    
            data_str += ' '.join(classes[obj].split('_')[1:]) + '\n'
            vals = [val_func(np.array(data_dict[k][obj])) for k in keys]
            bold_val = bold_func(vals)
            for v in vals:
                if(v == bold_val):
                    data_str += ' & \\textbf{' + '{:0.2f}'.format(v) + '}'
                else:
                    data_str += ' & {:0.2f}'.format(v)
            data_str += ' \\\\'
            data_str += '\n'

        data_str += '\hline\n'

    data_str += '{} \n'.format(agg_title)
    vals = [val_func(np.concatenate([np.array(data_dict[k][o]) for o in objs])) for k in keys]
    bold_val = bold_func(vals)
    for v in vals:
        if(v == bold_val):
            data_str += ' & \\textbf{' + '{:0.2f}'.format(v) + '}'
        else:
            data_str += ' & {:0.2f}'.format(v)
    data_str += ' \\\\'
    data_str += '\n'
    #print(data_str)
    return data_str
    
def makeTableHeading(names):
    data_str = ''
    data_str +='\\begin{table*}[h!]\n'
    data_str +='\\small\n'
    data_str +='\\centering\n'
    data_str +='\\begin{tabular} {||c||' + len(names)*'c|' +'|}\n'
    data_str +='\\hline\n'
        
    data_str += ' '
    for k in names:
        data_str += ' & ' + k
    data_str += '\\\\'
    data_str += '\n'

    data_str += '\hline\n'
    
    #print(data_str)
    return data_str

def makeTableEnding(caption = None, label = None):
    data_str = ''
    data_str +='\\hline\n'
    data_str +='\\end{tabular}\n'
    if(caption is not None):
        data_str +='\\caption{' + caption + '}\n'
    if(label is not None):
        data_str +='\\label{tbl:' + label + '}\n'
    data_str +='\\end{table*}\n'
    #print(data_str)
    return data_str

In [34]:
hist_global_results = 'ral_results/single_img_hist_orig_results.npz'
hist_conf_results = 'ral_results/single_img_hist_conf_orig_results.npz'
bing_fixed_results = 'ral_results/single_img_bing_results.npz'

results = {}
updateResults(results, hist_global_results)
updateResults(results, hist_conf_results)
updateResults(results, bing_fixed_results)

for res_k, res_v in results.items():
    for alg_k, alg_v in res_v.items():
        for k, v in alg_v.items():
            if(type(v) is dict):
                results[res_k][alg_k][k] = list(v.values())

In [59]:
all_item_tables = {}
from object_pose_utils.utils.pose_error import accuracyAUC
from functools import partial

for k, v in results.items():
    table_str = ''
    table_str += makeTableHeading([h.replace('_', ' ') for h in v.keys()])
    if(k == 'likelihood'):
        table_str += makeTableEntries(v, val_func=mean_log_clean, bold_func = max, individual = True)
    elif('angular' in k):
        table_str += makeTableEntries(v, val_func=mean_abs, bold_func = min, individual = True)    
        #table_str += makeTableEntries(v, val_func=partial(accuracyAUC, max_theshold=180), 
        #                              bold_func = max, individual = True)
    else:
        table_str += makeTableEntries(v, val_func=mean_abs, bold_func = min, individual = True)
        #table_str += makeTableEntries(v, val_func=partial(accuracyAUC, max_theshold=0.1), 
        #                              bold_func = max, individual = True)
    #table_str += makeTableEnding(caption = 'AUC ' + k.replace('_', ' '))
    table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
    all_item_tables[k] = table_str
    print(table_str)
    print('\n\n\n')


\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c||}
\hline
  & hist reg global & hist comp global & hist conf & bing fixed & bing mixed\\
\hline
master chef can
 & -2.82 & \textbf{1.01} & -0.69 & -0.97 & -0.94 \\
cracker box
 & -1.00 & 2.26 & \textbf{4.03} & 1.13 & 1.68 \\
sugar box
 & 2.64 & 2.24 & \textbf{3.81} & 0.04 & 0.97 \\
tomato soup can
 & -2.24 & \textbf{1.61} & 0.96 & -0.27 & -0.02 \\
mustard bottle
 & -1.21 & 2.15 & \textbf{3.86} & 2.38 & 2.41 \\
tuna fish can
 & -5.86 & \textbf{0.42} & -2.56 & -1.17 & -2.19 \\
pudding box
 & -0.18 & -1.35 & \textbf{1.99} & 1.15 & 1.19 \\
gelatin box
 & 2.48 & 2.08 & \textbf{4.67} & 1.33 & 1.92 \\
potted meat can
 & -1.36 & 0.61 & \textbf{1.58} & -0.01 & 0.03 \\
banana
 & -1.12 & 1.03 & \textbf{1.98} & -0.65 & -0.45 \\
pitcher base
 & 2.13 & 2.28 & \textbf{3.29} & 2.82 & 3.11 \\
bleach cleanser
 & -4.19 & 1.76 & \textbf{3.88} & 1.56 & 2.42 \\
bowl
 & -8.88 & -0.46 & \textbf{0.90} & -2.45 & -2.33 \\
mug
 & -6.18 & -1.03 &

In [58]:
all_item_tables = {}
from object_pose_utils.utils.pose_error import accuracyAUC
from functools import partial

for k, v in results.items():
    table_str = ''
    table_str += makeTableHeading([h.replace('_', ' ') for h in v.keys()])
    if(k == 'likelihood'):
        table_str += makeTableEntries(v, objs = non_sym_objs, 
                                      val_func=mean_log_clean, bold_func = max,
                                      agg_title = 'Non-Symmetric')
        table_str += makeTableEntries(v, objs = sym_objs,
                                      val_func=mean_log_clean, bold_func = max, 
                                      agg_title = 'Symmetric')
        table_str += makeTableEntries(v, val_func=mean_log_clean, bold_func = max)
    elif('angular' in k):
        table_str += makeTableEntries(v, objs = non_sym_objs, 
                                      val_func=mean_abs, bold_func = min,
                                      agg_title = 'Non-Symmetric')
        table_str += makeTableEntries(v, objs = sym_objs, 
                                      val_func=mean_abs, bold_func = min, 
                                      agg_title = 'Symmetric')
        table_str += makeTableEntries(v, val_func=mean_abs, bold_func = min)
        #table_str += makeTableEntries(v, val_func=partial(accuracyAUC, max_theshold=180), 
        #                              bold_func = max, individual = True)
    else:
        table_str += makeTableEntries(v, objs = non_sym_objs,
                                      val_func=mean_abs, bold_func = min,
                                      agg_title = 'Non-Symmetric')
        table_str += makeTableEntries(v, objs = sym_objs,
                                      val_func=mean_abs, bold_func = min,
                                      agg_title = 'Symmetric')
        table_str += makeTableEntries(v, val_func=mean_abs, bold_func = min)
        #table_str += makeTableEntries(v, val_func=partial(accuracyAUC, max_theshold=0.1), 
        #                              bold_func = max, individual = True)
    #table_str += makeTableEnding(caption = 'AUC ' + k.replace('_', ' '))
    table_str += makeTableEnding(caption = 'Mean ' + k.replace('_', ' '))
    all_item_tables[k] = table_str
    print(table_str)
    print('\n\n\n')

\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c||}
\hline
  & hist reg global & hist comp global & hist conf & bing fixed & bing mixed\\
\hline
Non-Symmetric 
 & -2.43 & 1.26 & \textbf{1.63} & 0.44 & 0.66 \\
Symmetric 
 & -11.44 & -5.93 & -2.91 & -2.34 & \textbf{-2.27} \\
All 
 & -3.90 & 0.09 & \textbf{0.89} & -0.01 & 0.18 \\
\hline
\end{tabular}
\caption{Mean likelihood}
\end{table*}





\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c||}
\hline
  & hist reg global & hist comp global & hist conf & bing fixed & bing mixed\\
\hline
Non-Symmetric 
 & 21.74 & 21.74 & 21.74 & \textbf{21.72} & \textbf{21.72} \\
Symmetric 
 & \textbf{71.44} & \textbf{71.44} & \textbf{71.44} & 71.47 & 71.47 \\
All 
 & 29.86 & 29.86 & 29.86 & \textbf{29.85} & \textbf{29.85} \\
\hline
\end{tabular}
\caption{Mean sym angular error}
\end{table*}





\begin{table*}[h!]
\small
\centering
\begin{tabular} {||c||c|c|c|c|c||}
\hline
  & hist reg global & hist comp global & his

In [35]:
old_hist_results = 'ral_results/single_img_results.npz'
old_results = {}
updateResults(old_results, old_hist_results)

In [61]:
print(mean_log_clean(results['likelihood']['hist_conf'][19]))
print(mean_log_clean(results['likelihood']['hist_comp_global'][19]))


-5.550682973753043
-10.450504445126201


In [68]:
np.log(results['likelihood']['hist_comp_global'][19])

array([-2.76398614e+01, -1.17265607e+01, -1.05128298e+01, -9.56166938e+00,
       -1.03226954e+01, -1.30577700e+01, -3.15368584e+01, -2.82804491e+01,
       -2.39361721e+01, -1.15214879e+01, -2.75959299e+01, -2.88752009e+01,
       -2.81989687e+01, -2.73482227e+01, -3.30842368e+01, -2.95222406e+01,
       -1.61589364e+01, -1.68491082e+01, -1.65632775e+01, -2.41959320e+01,
       -1.67768797e+01, -1.29167911e+01, -1.42840955e+01, -2.22151622e+01,
       -1.89400312e+01, -2.05515898e+01, -1.60218617e+01, -1.22245653e+01,
       -1.73861522e+01, -1.50851754e+01, -1.68272508e+01, -1.62041168e+01,
       -3.25336002e+01, -2.06854247e+01, -2.59648622e+01, -2.38920554e+01,
       -1.23620870e+01, -2.13059571e+01, -3.48734111e+01, -3.46345636e+01,
       -1.01829968e+01, -2.57339772e+01, -2.80193338e+01, -2.25879527e+01,
       -3.51070262e+01, -3.36519326e+01, -2.39199952e+01, -4.34715733e+00,
       -4.53029025e+01, -3.78957040e+01, -1.10287235e+01, -1.13765045e+01,
       -1.31321557e+01, -